# **⚡ SAM Algorithm**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## **🧮 Intermolecular Potential**

In [2]:
# Lennard-Jones potential between all pairs
def intermolecular_pair_potential(coordinates, epsilon=1.0, sigma=1.0):
    total_energy = 0.0
    n = len(coordinates)
    for i in range(n - 1):
        for j in range(i + 1, n):
            r = np.linalg.norm(coordinates[i] - coordinates[j])
            # Avoid division by zero or too small r
            if r == 0:
                continue
            total_energy += 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)
    return total_energy

## **🧬 Annealing Lennard-Jones**

In [ ]:
def adaptive_simulated_annealing_per_dimension(initial_coords, epsilon=1.0, sigma=1.0,
                                               initial_step=1e-4, initial_temp=1.0, max_iter_factor=10000000):
    t_steps = max_iter_factor / 10
    pm = 0.04
    n_particles, dims = initial_coords.shape
    max_iterations = max_iter_factor * n_particles * 3

    coords = initial_coords.copy()
    current_energy = intermolecular_pair_potential(coords, epsilon, sigma)
    best_coords = coords.copy()
    best_energy = current_energy

    T = initial_temp
    step_matrix = 1e-4
    accept_matrix = 0.0

    adapt_interval = 100000
    step_history = []

    for iteration in range(1, max_iterations + 1):
        
        if iteration % t_steps == 0: T *= 0.9

        proposal = np.copy(coords)
        for i in range(n_particles):
            noise = np.random.normal(0.0)
            for j in range(3):
                if np.random.rand() > pm: noise = 0.0
                proposal[i][j] += noise
        
        proposal_energy = intermolecular_pair_potential(proposal, epsilon, sigma)

        if proposal_energy < current_energy:
            coords = proposal
            current_energy = proposal_energy
            accept_matrix += 1

            if current_energy < best_energy:
                best_energy = current_energy
                best_coords = coords.copy()

        if iteration % adapt_interval == 0:
            acceptance_rate = accept_matrix / adapt_interval

            if np.random.rand() > 0.5:
                if acceptance_rate < 0.001:
                    step_matrix *= 0.9
                elif acceptance_rate > 0.002:
                    step_matrix *= 1.1
            else:
                if acceptance_rate < 0.001:
                    pm *= 0.9
                elif acceptance_rate > 0.002:
                    pm *= 1.1

            if pm > 0.5: pm = 0.5
            if pm < 1.0 / (n_particles * 3.0): pm = 1.0 / (n_particles * 3.0)

            if step_matrix > 100.0: step_matrix = 100.0
            if step_matrix < 1e-8: step_matrix = 1e-8

            accept_matrix = 0.0

            

            avg_step = np.mean(step_matrix)
            step_history.append((iteration, avg_step))
            print(f"Iter {iteration:6d} | Temp {T:.5f} | Step Matrix {step_matrix:.5f} | Best Energy {best_energy:.5f} | PM {pm:.5} | Acceptance Rate {acceptance_rate}")

    print(f"Best energy found: {best_energy:.5f}")
    return best_coords, best_energy

## **🧪 Testing**

In [59]:
# Example usage
initial_coords = np.random.rand(10, 3)  # 10 particles in 3D

optimized_coords, final_energy = adaptive_simulated_annealing_per_dimension(initial_coords)

print("Optimized Coordinates:\n", optimized_coords)
print("Final Potential Energy:", final_energy)

Iter  10000 | Temp 1.00000 | Step Matrix 0.00010 | Best Energy -17.18607 | PM 0.036 | Acceptance Rate 0.0074
Iter  20000 | Temp 1.00000 | Step Matrix 0.00010 | Best Energy -17.50734 | PM 0.033333 | Acceptance Rate 0.002
Iter  30000 | Temp 1.00000 | Step Matrix 0.00010 | Best Energy -17.51739 | PM 0.033333 | Acceptance Rate 0.0007
Iter  40000 | Temp 1.00000 | Step Matrix 0.00009 | Best Energy -17.60756 | PM 0.033333 | Acceptance Rate 0.0007
Iter  50000 | Temp 1.00000 | Step Matrix 0.00008 | Best Energy -18.04034 | PM 0.033333 | Acceptance Rate 0.002
Iter  60000 | Temp 1.00000 | Step Matrix 0.00008 | Best Energy -18.04981 | PM 0.033333 | Acceptance Rate 0.0003
Iter  70000 | Temp 1.00000 | Step Matrix 0.00008 | Best Energy -19.28168 | PM 0.033333 | Acceptance Rate 0.0026
Iter  80000 | Temp 1.00000 | Step Matrix 0.00007 | Best Energy -19.78436 | PM 0.033333 | Acceptance Rate 0.0025
Iter  90000 | Temp 1.00000 | Step Matrix 0.00007 | Best Energy -19.81531 | PM 0.033333 | Acceptance Rate 0.00

KeyboardInterrupt: 